# Cuaderno de Documentación: 04 - Componentes RAG (`rag_components.py`)

El módulo `scripts/rag_components.py` es donde se ensamblan los componentes principales del sistema de Generación Aumentada por Recuperación (RAG). Esto incluye la inicialización del Modelo de Lenguaje Grande (LLM), la definición de la plantilla de prompt que guiará al LLM, y la creación de la cadena `RetrievalQA` que orquesta la recuperación de información y la generación de la respuesta.

## Funcionalidades Principales

1.  **`PROMPT_TEMPLATE_STR` (Constante):**
    * **Propósito:** Define la plantilla de instrucciones que se enviará al LLM. Esta plantilla es crucial para dirigir el comportamiento del modelo y asegurar que la respuesta se genere en el formato y con el contenido deseado.
    * **Contenido Típico:**
        * **Rol del Asistente:** Define la personalidad o especialización del LLM (ej. "asistente de IA especializado en identificación de riesgos...").
        * **Instrucciones de Tarea:** Describe qué se espera que haga el LLM (ej. analizar un nuevo proyecto, usar solo el contexto proporcionado).
        * **Marcadores de Posición:**
            * `{context}`: Donde se insertarán los fragmentos de texto recuperados de la base de datos vectorial.
            * `{question}`: Donde se insertará la descripción del nuevo proyecto (la consulta del usuario).
        * **Formato de Salida:** Especifica cómo debe estructurarse la respuesta (ej. "Formatea TODA tu respuesta como un ÚNICO objeto JSON..." con un ejemplo de la estructura JSON esperada).

2.  **`get_llm_instance(model_name, gemini_api_key)`:**
    * **Propósito:** Inicializar y devolver una instancia del LLM, en este caso, un modelo de Google Gemini.
    * **Proceso:**
        * Utiliza `ChatGoogleGenerativeAI` de LangChain para interactuar con los modelos de Gemini.
        * Requiere el `model_name` (ej. `gemini-1.5-flash-latest` de `config.py`) y la `gemini_api_key`.
        * Se puede configurar la `temperature` (un valor bajo como 0.3 produce respuestas más deterministas y factuales, adecuado para análisis de riesgos).
        * La opción `convert_system_message_to_human` puede ser necesaria para la compatibilidad con algunos formatos de mensaje de los modelos Gemini, aunque se recomienda verificar la documentación más reciente de LangChain para esta opción.
    * **Retorno:** Una instancia del LLM configurado, o `None` si hay un error.

3.  **`crear_cadena_rag(llm, vector_db_instance, k_retrieved_docs)`:**
    * **Propósito:** Construir y devolver la cadena `RetrievalQA`, que es el núcleo del sistema RAG.
    * **Proceso:**
        * **Retriever:** Crea un `retriever` a partir de la `vector_db_instance` (la base de datos ChromaDB). El retriever se configura para buscar los `k_retrieved_docs` fragmentos más similares semánticamente a la consulta del usuario.
        * **Prompt Template:** Utiliza la `PROMPT_TEMPLATE_STR` para crear un objeto `PromptTemplate` de LangChain.
        * **Ensamblaje de `RetrievalQA`:** Se utiliza `RetrievalQA.from_chain_type` para crear la cadena.
            * `llm`: La instancia del LLM configurada.
            * `chain_type="stuff"`: Este es el tipo de cadena más simple. Toma todos los documentos recuperados y los "amontona" (stuff) en el contexto del prompt. Es adecuado si el contexto combinado no excede el límite de tokens del LLM.
            * `retriever`: El retriever configurado previamente.
            * `return_source_documents=True`: Indica a la cadena que devuelva también los documentos fuente que se utilizaron para generar la respuesta. Esto es útil para la trazabilidad y la verificación.
            * `chain_type_kwargs={"prompt": prompt}`: Pasa la plantilla de prompt personalizada a la cadena.
    * **Retorno:** Una instancia de la cadena `RetrievalQA` lista para ser invocada, o `None` si hay un error.

A continuación, se muestra el contenido del script `rag_components.py` para referencia.

In [ ]:
# Cargar el contenido del script rag_components.py
%load ../scripts/rag_components.py